# The QR Decomposition 

The $QR$ decomposition is a fundamentally important matrix factorization. It
is straightforward to implement, is numerically stable, and provides the 
basis of several important algorithms. In this lab we explore several 
ways to produce the QR decomposition and implement a few immediate 
applications.

The QR decomposition of a matrix $A$ is a factoration 

$$A =QR$$ 

where $Q$ has orthonormal columns and $R$ is upper triangular. Every $m \times n$
matrix $A$ of rank $n \leq m$ has a QR decomposition, with two main forms. 

- Reduced QR: Q is $m \times n$, R is $n \times n$, and the columns $\{q_j\}_{j=1}^m$ 
of Q form an orthonormal basis of $A$ 

-Full QR: Q is $m \times m$, R is $m \times n$. In this case, the columns
$\{q_j\}_{j=1}^m$ of Q form an orthonormal basis for all of $F^m$, and the last $m-n$ rows of R only contain zeros. 

We distinguish between these two forms by writing $\hat{Q}$ and $\hat{R}$
for reduced decomposition and Q and R for the full decomposition. 

![QR](../images/QR-decomposition.png)

## QR via Gram-Schmidt 